In [1]:
import asyncio
import collections
import contextlib

import aiohttp
from aiohttp import web
import tqdm

%run flags2_common.ipynb import main, HTTPStatus, Result, save_flag

In [2]:
DEFAULT_CONCUR_REQ = 5
MAX_CONCUR_REQ = 1000

In [3]:
class FetchError(Exception):
    def __init__(self, country_code):
        self.country_code = country_code

In [4]:
@asyncio.coroutine
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = yield from aiohttp.request('GET', url)
    with contextlib.closing(resp):
        if resp.status == 200:
            image = yield from resp.read()
            return image
        elif resp.status == 404:
            raise web.HTTPNotFound()
        else:
            raise aiohttp.HttpProcessingError(
                code=resp.status, message=resp.reason,
                headers=resp.headers)

C:\Users\Ivo\AppData\Local\Temp\ipykernel_5644\2245717180.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def get_flag(base_url, cc):


In [5]:
@asyncio.coroutine
def download_one(cc, base_url, semaphore, verbose):
    try:
        with (yield from semaphore):
            image = yield from get_flag(base_url, cc)
    except web.HTTPNotFound:
        status = HTTPStatus.not_found
        msg = 'not found'
    except Exception as exc:
        raise FetchError(cc) from exc
    else:
        loop = asyncio.get_event_loop()  # <1>
        loop.run_in_executor(None,  # <2>
                save_flag, image, cc.lower() + '.gif')  # <3>
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose and msg:
        print(cc, msg)

    return Result(status, cc)

C:\Users\Ivo\AppData\Local\Temp\ipykernel_5644\1807842025.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def download_one(cc, base_url, semaphore, verbose):


In [6]:
@asyncio.coroutine
def downloader_coro(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    semaphore = asyncio.Semaphore(concur_req)
    to_do = [download_one(cc, base_url, semaphore, verbose)
             for cc in sorted(cc_list)]

    to_do_iter = asyncio.as_completed(to_do)
    if not verbose:
        to_do_iter = tqdm.tqdm(to_do_iter, total=len(cc_list))
    for future in to_do_iter:
        try:
            res = yield from future
        except FetchError as exc:
            country_code = exc.country_code
            try:
                error_msg = exc.__cause__.args[0]
            except IndexError:
                error_msg = exc.__cause__.__class__.__name__
            if verbose and error_msg:
                msg = '*** Error for {}: {}'
                print(msg.format(country_code, error_msg))
            status = HTTPStatus.error
        else:
            status = res.status

        counter[status] += 1

    return counter

C:\Users\Ivo\AppData\Local\Temp\ipykernel_5644\1040380535.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def downloader_coro(cc_list, base_url, verbose, concur_req):


In [7]:
def download_many(cc_list, base_url, verbose, concur_req):
    loop = asyncio.get_event_loop()
    coro = downloader_coro(cc_list, base_url, verbose, concur_req)
    counts = loop.run_until_complete(coro)
    loop.close()

    return counts

In [8]:
if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)

usage: ipykernel_launcher.py [-h] [-a] [-e] [-l N] [-m CONCURRENT] [-s LABEL] [-v] [CC ...]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\Ivo\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
